In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [69.5 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-10-23 12:34:16--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.52MB/s    in 0.2s    

2021-10-23 12:34:16 (4.52 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("watchesReviews").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
path = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz'
spark.sparkContext.addFile(path)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv.gz"), sep="\t", header=True)
df.show(5)

In [ ]:
df.count()

In [ ]:
df.printSchema()

In [ ]:
from pyspark.sql.types import IntegerType
df = df.withColumn("customer_id", df["customer_id"].cast(IntegerType()))
df = df.withColumn("product_parent", df["product_parent"].cast(IntegerType()))
df = df.withColumn("star_rating", df["star_rating"].cast(IntegerType()))
df = df.withColumn("helpful_votes", df["helpful_votes"].cast(IntegerType()))
df = df.withColumn("total_votes", df["total_votes"].cast(IntegerType()))

In [ ]:

df.printSchema()

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
df = df.withColumn("review_date", to_date(df["review_date"],"yyyy-mm-dd") )
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [ ]:
review_id_df = df.select(["review_id","customer_id","product_id","product_parent","review_date"]).na.drop(subset=["review_id"])
products_df = df.select(["product_id","product_title"]).na.drop(subset=["product_id"])
customers_df = df.groupBy("customer_id").count().withColumnRenamed("count", "customer_count").na.drop(subset=["customer_id"])
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine"]).na.drop(subset=["review_id"])

In [ ]:
mode = "append"
jdbc_url="jdbc:postgresql://bootcamp-amazon-reviews.cs94zx5i7cgb.us-east-2.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", 
          "password": "postgres", 
          "driver":"org.postgresql.Driver"}

In [ ]:
review_id_df = review_id_df.dropDuplicates(['review_id'])
products_df = products_df.dropDuplicates(['product_id'])
customers_df = customers_df.dropDuplicates(['customer_id'])
vine_df = vine_df.dropDuplicates(['review_id'])

In [ ]:
table = "review_id_table"
review_id_df.write.jdbc(url=jdbc_url, table=table, mode=mode, properties=config)




In [ ]:
table = "products"
products_df.write.jdbc(url=jdbc_url, table=table, mode=mode, properties=config)

In [ ]:
table = "customers"
customers_df.write.jdbc(url=jdbc_url, table=table, mode=mode, properties=config)

In [ ]:
table = "vine_table"
vine_df.write.jdbc(url=jdbc_url, table=table, mode=mode, properties=config)